In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import time
import math
import pandas as pd

### web-scraping code

In [3]:
# CHOOSE AN OPTION´: "apartments" or 'houses'

properties_to_look_for = "houses"

In [4]:
def url_selection(properties_to_look_for):

    if properties_to_look_for == 'apartments':
        url = 'https://www.inmuebles24.com/departamento-compartido-o-departamentos-o-desarrollo-vertical-en-venta-en-distrito-federal-q-ciudad-de-mexico.html'
#         url_several = f'https://www.inmuebles24.com/departamento-compartido-o-departamentos-o-desarrollo-vertical-en-venta-en-distrito-federal-pagina-{i}-q-ciudad-de-mexico.html'

    elif properties_to_look_for == 'houses':
        url = 'https://www.inmuebles24.com/casas-o-duplex-o-casa-en-condominio-o-casa-uso-de-suelo-o-desarrollo-horizontal-en-venta-en-distrito-federal-q-ciudad-de-mexico.html'
#         url_several = f'https://www.inmuebles24.com/casas-o-duplex-o-casa-en-condominio-o-casa-uso-de-suelo-o-desarrollo-horizontal-en-venta-en-distrito-federal-pagina-{i}-q-ciudad-de-mexico.html'

    return (url)

# url_selection(properties_to_look_for)

In [5]:
# MULTIPLE PAGES:

url = url_selection(properties_to_look_for)
    
count_number = 0

permalink_list = []
titles_list = []
price_list = []
bedrooms_number_list = []
full_bathrooms_list = []
thumbnail_list = []
announcement_id_list = []
address_street_list = []
colonia_list = []
city_list = []
state_list = []
property_type_list = []
type_of_operation_list = []
currency_list = []
real_state_agency_list = []

properties_feature_list = []  
        

executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)

browser.visit(url)
browser.is_element_not_present_by_id('list-card-container', wait_time=2)
html = browser.html
soup = bs(html, "html.parser")

number_of_results = soup.find("h1", class_ = "list-result-title").text.split(" ")[0].replace(',','')
number_of_results = int(number_of_results)

total_paging_times = math.ceil(number_of_results/24) + 1
print(F'properties_to_look_for: {properties_to_look_for}, total_paging_times: {total_paging_times}')
print('')


for i in range(1,total_paging_times):   

    
#   HERE IS WHERE I PUT THE URL
    if i == 1:
        url = url
    else:
        if properties_to_look_for == 'apartments':
            url = f'https://www.inmuebles24.com/departamento-compartido-o-departamentos-o-desarrollo-vertical-en-venta-en-distrito-federal-pagina-{i}-q-ciudad-de-mexico.html'

        elif properties_to_look_for == 'houses':
            url = f'https://www.inmuebles24.com/casas-o-duplex-o-casa-en-condominio-o-casa-uso-de-suelo-o-desarrollo-horizontal-en-venta-en-distrito-federal-pagina-{i}-q-ciudad-de-mexico.html'

    
    
    print(f'URL VISITED: {url}')
    print('')
    
    browser.visit(url)
    browser.is_element_not_present_by_id('list-card-container', wait_time=2)
    html = browser.html
    soup = bs(html, "html.parser")
    
    publications_list = soup.find_all("div", class_ = "postingCard")

    for x in range(len(publications_list)):

        count_number += 1

    #     PROPERTY_TYPE:

        if 'departamento' in url:
            property_type = 'Departamento'
        elif 'casa' in url:
            property_type = 'Casa'

        property_type_list.append(property_type)

    #     TYPE OF OPERATION:

        if 'venta' in url:
            type_of_operation = 'Venta'
        elif 'renta' in url:
            type_of_operation = 'Renta'

        type_of_operation_list.append(type_of_operation)

    #   PROPERTY_ID
        property_id = publications_list[x]["data-id"]
        announcement_id_list.append(property_id)


    #     HYPERLINK    
        hyperlink = f'https://www.inmuebles24.com{publications_list[x]["data-to-posting"]}'
        permalink_list.append(hyperlink)

    #     ------SUB_SOUP-----

        sub_soup = publications_list[x]

    #     PROPERTY_TITLE
        ad_title = sub_soup.find_all("h2", class_ = "postingCardTitle")[0].get_text(strip=True)
        titles_list.append(ad_title)

    #     PROPERTY PRICE
        property_price = sub_soup.find_all("span", class_ = "firstPrice")[0].get_text(strip=True).replace('MN ', '').replace(',', '')
        price_list.append(property_price)

    #     CURRENCY:

        if 'USD' in property_price:
            currency = 'USD'
        else:
            currency = 'MXN'

        currency_list.append(currency)

    # #     print(f'id: {property_id} , title: {ad_title} , {hyperlink} , price: {property_price}')

    #     THUMBNAIL:

        try:
            thumbnail = sub_soup.find_all("img")[0]['data-src'].strip()
            thumbnail_list.append(thumbnail)
        except:
            thumbnail = sub_soup.find_all("img")[0]['src'].strip()
            thumbnail_list.append(thumbnail)


        #   LOCATION TITLE

        street = sub_soup.find_all("span", class_ = "postingCardLocationTitle")[0].get_text(strip=True)

        address = sub_soup.find_all("span", class_ = "postingCardLocation")[0].get_text(strip=True).split(",")
        colonia = address[0]
        city = address[1]
        state = 'Distrito Federal'

        address_street_list.append(street)
        colonia_list.append(colonia)
        city_list.append(city)
        state_list.append(state)

    #     REAL ESTATE AGENCY:
        
        try:
            real_state_agency = sub_soup.find_all("div", class_ = "publisherLogo")[0].a.img['src'].split("/")[-1]

            if real_state_agency == 'no-logo.svg':
                real_state_agency = 'trato_directo'
            else:
                real_state_agency = real_state_agency.replace('.jpg', '').replace('logo_', '')
        
        except:
            real_state_agency = 'not_available'

        real_state_agency_list.append(real_state_agency)

    #     FEATURES LIST:

        inside_list = []

        features_list = sub_soup.find_all("ul", class_ = "postingCardMainFeatures")[0].find_all("li")

        for j in range(len(features_list)):
            feature = features_list[j].get_text(strip = True).replace("\n", " ").replace("\t", "")
            inside_list.append(feature)

    #         BEDROOM NUMBER:

            if 'rec.' in feature:
                bedrooms_number = feature[0:2]
                bedrooms_number_list.append(bedrooms_number)
                
    #         BATHROOMS NUMBER:

            if 'baños' in feature or 'baño' in feature:
                bathrooms_number = feature[0:2]
                full_bathrooms_list.append(bathrooms_number)
                
        if len(bedrooms_number_list) != count_number:
            bedrooms_number_list.append("not_available")

        if len(full_bathrooms_list) != count_number:
            full_bathrooms_list.append("not_available")

        properties_feature_list.append(inside_list)

        print(f"""Property_Number: {count_number} - property_id: {property_id}
        """)
        
    time.sleep(2.5)

    #   street: {street}, colonia: {colonia}, city: {city}, state: {state},      
#     ad_title: {ad_title}, property_price: {property_price},
    #     thumbnail: {thumbnail} bedrooms_number: {bedrooms_number}, bathrooms_number: {bathrooms_number},
    #     real_state_agency: {real_state_agency}

[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\Intel NUC\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


 
properties_to_look_for: houses, total_paging_times: 35

URL VISITED: https://www.inmuebles24.com/casas-o-duplex-o-casa-en-condominio-o-casa-uso-de-suelo-o-desarrollo-horizontal-en-venta-en-distrito-federal-q-ciudad-de-mexico.html

Property_Number: 1 - property_id: 60924386
        
Property_Number: 2 - property_id: 60895016
        
Property_Number: 3 - property_id: 55875877
        
Property_Number: 4 - property_id: 59712976
        
Property_Number: 5 - property_id: 60426612
        
Property_Number: 6 - property_id: 60264102
        
Property_Number: 7 - property_id: 60219412
        
Property_Number: 8 - property_id: 59596560
        
Property_Number: 9 - property_id: 58133740
        
Property_Number: 10 - property_id: 59823425
        
Property_Number: 11 - property_id: 59598685
        
Property_Number: 12 - property_id: 58491300
        
Property_Number: 13 - property_id: 55392439
        
Property_Number: 14 - property_id: 59856085
        
Property_Number: 15 - property_id:

URL VISITED: https://www.inmuebles24.com/casas-o-duplex-o-casa-en-condominio-o-casa-uso-de-suelo-o-desarrollo-horizontal-en-venta-en-distrito-federal-pagina-7-q-ciudad-de-mexico.html

Property_Number: 145 - property_id: 59718328
        
Property_Number: 146 - property_id: 59548642
        
Property_Number: 147 - property_id: 59548455
        
Property_Number: 148 - property_id: 59548205
        
Property_Number: 149 - property_id: 57050915
        
Property_Number: 150 - property_id: 59522376
        
Property_Number: 151 - property_id: 59519581
        
Property_Number: 152 - property_id: 59488625
        
Property_Number: 153 - property_id: 59488485
        
Property_Number: 154 - property_id: 59447531
        
Property_Number: 155 - property_id: 59447299
        
Property_Number: 156 - property_id: 59447178
        
Property_Number: 157 - property_id: 59447065
        
Property_Number: 158 - property_id: 59437888
        
Property_Number: 159 - property_id: 59437339
        
Proper

URL VISITED: https://www.inmuebles24.com/casas-o-duplex-o-casa-en-condominio-o-casa-uso-de-suelo-o-desarrollo-horizontal-en-venta-en-distrito-federal-pagina-13-q-ciudad-de-mexico.html

Property_Number: 289 - property_id: 60336636
        
Property_Number: 290 - property_id: 59419976
        
Property_Number: 291 - property_id: 60034533
        
Property_Number: 292 - property_id: 59796783
        
Property_Number: 293 - property_id: 59561317
        
Property_Number: 294 - property_id: 59306571
        
Property_Number: 295 - property_id: 59085553
        
Property_Number: 296 - property_id: 58620905
        
Property_Number: 297 - property_id: 57118517
        
Property_Number: 298 - property_id: 56432968
        
Property_Number: 299 - property_id: 59697643
        
Property_Number: 300 - property_id: 60436739
        
Property_Number: 301 - property_id: 57645957
        
Property_Number: 302 - property_id: 57123420
        
Property_Number: 303 - property_id: 56733316
        
Prope

URL VISITED: https://www.inmuebles24.com/casas-o-duplex-o-casa-en-condominio-o-casa-uso-de-suelo-o-desarrollo-horizontal-en-venta-en-distrito-federal-pagina-19-q-ciudad-de-mexico.html

Property_Number: 433 - property_id: 59216870
        
Property_Number: 434 - property_id: 59116081
        
Property_Number: 435 - property_id: 58976236
        
Property_Number: 436 - property_id: 57123526
        
Property_Number: 437 - property_id: 57122790
        
Property_Number: 438 - property_id: 56733439
        
Property_Number: 439 - property_id: 56731807
        
Property_Number: 440 - property_id: 56720806
        
Property_Number: 441 - property_id: 56720664
        
Property_Number: 442 - property_id: 56718833
        
Property_Number: 443 - property_id: 56712499
        
Property_Number: 444 - property_id: 56711561
        
Property_Number: 445 - property_id: 56693411
        
Property_Number: 446 - property_id: 56693135
        
Property_Number: 447 - property_id: 56688491
        
Prope

URL VISITED: https://www.inmuebles24.com/casas-o-duplex-o-casa-en-condominio-o-casa-uso-de-suelo-o-desarrollo-horizontal-en-venta-en-distrito-federal-pagina-25-q-ciudad-de-mexico.html

Property_Number: 577 - property_id: 56718614
        
Property_Number: 578 - property_id: 56718521
        
Property_Number: 579 - property_id: 56713361
        
Property_Number: 580 - property_id: 56711172
        
Property_Number: 581 - property_id: 56694049
        
Property_Number: 582 - property_id: 56683837
        
Property_Number: 583 - property_id: 56610918
        
Property_Number: 584 - property_id: 56608733
        
Property_Number: 585 - property_id: 56595911
        
Property_Number: 586 - property_id: 56595636
        
Property_Number: 587 - property_id: 56592017
        
Property_Number: 588 - property_id: 56591758
        
Property_Number: 589 - property_id: 56434237
        
Property_Number: 590 - property_id: 56396064
        
Property_Number: 591 - property_id: 56395072
        
Prope

URL VISITED: https://www.inmuebles24.com/casas-o-duplex-o-casa-en-condominio-o-casa-uso-de-suelo-o-desarrollo-horizontal-en-venta-en-distrito-federal-pagina-31-q-ciudad-de-mexico.html

Property_Number: 721 - property_id: 60302157
        
Property_Number: 722 - property_id: 56127720
        
Property_Number: 723 - property_id: 60314886
        
Property_Number: 724 - property_id: 60207177
        
Property_Number: 725 - property_id: 60629138
        
Property_Number: 726 - property_id: 60627783
        
Property_Number: 727 - property_id: 60618360
        
Property_Number: 728 - property_id: 60613534
        
Property_Number: 729 - property_id: 60579695
        
Property_Number: 730 - property_id: 60607360
        
Property_Number: 731 - property_id: 60426821
        
Property_Number: 732 - property_id: 60601499
        
Property_Number: 733 - property_id: 60580291
        
Property_Number: 734 - property_id: 60563073
        
Property_Number: 735 - property_id: 60560420
        
Prope

### Debugger code

In [17]:
fields = [announcement_id_list,
    titles_list,
    type_of_operation_list,
    property_type_list,
    price_list,
    currency_list,
    bedrooms_number_list,
    full_bathrooms_list,
    real_state_agency_list,
    permalink_list,
    thumbnail_list,
    address_street_list,
    colonia_list,
    city_list,
    state_list]


for x in range(len(fields)):
    print(f'field {x+1}, length: {len(fields[x])}')

field 1, length: 813
field 2, length: 813
field 3, length: 813
field 4, length: 813
field 5, length: 813
field 6, length: 813
field 7, length: 813
field 8, length: 813
field 9, length: 813
field 10, length: 813
field 11, length: 813
field 12, length: 813
field 13, length: 813
field 14, length: 813
field 15, length: 813


### Create the table with the information scrapped

In [6]:
properties_24_df = pd.DataFrame({
    'ad_id': announcement_id_list,
    'title': titles_list,
    'type_of_operation': type_of_operation_list,
    'property_type': property_type_list,
    'price': price_list,
    "currency" : currency_list,
    'bedrooms_number': bedrooms_number_list,
    'full_bathrooms_number': full_bathrooms_list,
    'real_state_agency': real_state_agency_list,
    'permalink': permalink_list,
    'thumbnail': thumbnail_list,
    'address_street': address_street_list,
    'neighborhood': colonia_list,
    'city': city_list,
    'state': state_list,
})

properties_24_df.head()

,ad_id,title,type_of_operation,property_type,price,currency,bedrooms_number,full_bathrooms_number,real_state_agency,permalink,thumbnail,address_street,neighborhood,city,state
0,60924386,"Venta Casa Pedregal, Álvaro Obregón, Ciudad de...",Venta,Casa,32900000,MXN,4,4,maas-inmobiliaria_1527699791800,https://www.inmuebles24.com/propiedades/venta-...,https://img10.naventcdn.com/avisos/18/00/60/92...,Av las fuentes,Jardines del Pedregal,Alvaro Obregón,Distrito Federal
1,60895016,Magnífica Casa en Una de Las Mejores Colonias ...,Venta,Casa,USD 5500000,USD,3,3,coldwell-banker-invest-asociada-con-id-15246-7_2,https://www.inmuebles24.com/propiedades/magnif...,https://img10.naventcdn.com/avisos/18/00/60/89...,Sierra Gorda,Lomas de Chapultepec,Miguel Hidalgo,Distrito Federal
2,55875877,Vendo Casa en Lomas de Chapultepec Ciudad de M...,Venta,Casa,61400000,MXN,3,3,trato_directo,https://www.inmuebles24.com/propiedades/vendo-...,https://img10.naventcdn.com/avisos/18/00/55/87...,Lomas de Chapultepec,Lomas de Chapultepec,Miguel Hidalgo,Distrito Federal
3,59712976,"Residencia en Venta, del Carmen, Coyoacán, Ciu...",Venta,Casa,15180000,MXN,4,4,urbaunidad_1,https://www.inmuebles24.com/propiedades/reside...,https://img10.naventcdn.com/avisos/18/00/59/71...,Hidalgo,Del Carmen,Coyoacán,Distrito Federal
4,60426612,Casa en Venta en Cuajimalpa,Venta,Casa,7100000,MXN,4,3,placeholder-img.png,https://www.inmuebles24.com/propiedades/casa-e...,https://img10.naventcdn.com/avisos/18/00/60/42...,Cuajimalpa,Cuajimalpa,Cuajimalpa de Morelos,Distrito Federal


### Data Exploration and remove of duplicated records (if any)

In [7]:
properties_24_df.shape

(813, 15)

In [10]:
properties_24_df[properties_24_df.duplicated(subset=['ad_id'])]

,ad_id,title,type_of_operation,property_type,price,currency,bedrooms_number,full_bathrooms_number,real_state_agency,permalink,thumbnail,address_street,neighborhood,city,state


In [19]:
properties_24_df = properties_24_df.drop(properties_24_df[properties_24_df.duplicated(subset=['ad_id'])].index)

In [23]:
properties_24_df = properties_24_df.reset_index()

del properties_24_df['index']

properties_24_df.head(1)

,ad_id,title,type_of_operation,property_type,price,currency,bedrooms_number,full_bathrooms_number,real_state_agency,permalink,thumbnail,address_street,neighborhood,city,state
0,52913725,Bosque 6060,Venta,Departamento,11312978,MXN,3,not_available,abilia-creando-la-diferencia_1617205333633,https://www.inmuebles24.com/propiedades/bosque...,https://img10.naventcdn.com/listado/RPLISv6.16...,Desierto de los Léones 6060 Ciudad de México,Olivar de los Padres,Alvaro Obregón,Distrito Federal


In [136]:
browser.quit()

## ADD TIMESTAMP TO THE FINAL TABLE

In [12]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
 
# print("now =", now)

# dd/mm/YY H:M:S
dt_string = now.strftime("%m/%d/%Y")
print(dt_string)	

04/22/2021


In [13]:
properties_24_df.index = pd.Series([dt_string] * len(properties_24_df))
properties_24_df = properties_24_df.reset_index()
properties_24_df = properties_24_df.rename(columns={"index":"updated_at"})
properties_24_df.head(2)

,updated_at,ad_id,title,type_of_operation,property_type,price,currency,bedrooms_number,full_bathrooms_number,real_state_agency,permalink,thumbnail,address_street,neighborhood,city,state
0,04/22/2021,60924386,"Venta Casa Pedregal, Álvaro Obregón, Ciudad de...",Venta,Casa,32900000,MXN,4,4,maas-inmobiliaria_1527699791800,https://www.inmuebles24.com/propiedades/venta-...,https://img10.naventcdn.com/avisos/18/00/60/92...,Av las fuentes,Jardines del Pedregal,Alvaro Obregón,Distrito Federal
1,04/22/2021,60895016,Magnífica Casa en Una de Las Mejores Colonias ...,Venta,Casa,USD 5500000,USD,3,3,coldwell-banker-invest-asociada-con-id-15246-7_2,https://www.inmuebles24.com/propiedades/magnif...,https://img10.naventcdn.com/avisos/18/00/60/89...,Sierra Gorda,Lomas de Chapultepec,Miguel Hidalgo,Distrito Federal


## SAVE the Final result

In [16]:
# properties_24_df.to_csv('csv_files/inmuebles_24_CASAS_CDMX.csv', index = False, encoding = 'utf16')

In [ ]:
# tabla_final = tabla_final.sort_values(by = 'Número de m2', ascending = False)

In [ ]:
# tabla_final.to_json('static\data/ML_departamentos_CDMX.json', force_ascii = True, orient="records")

# Import the created file

In [18]:
import pandas as pd

In [19]:
propiedades_24_df = pd.read_csv('csv_files/inmuebles_24_CASAS_CDMX.csv', encoding = 'utf16')

In [20]:
propiedades_24_df.shape

(813, 16)

In [21]:
propiedades_24_df.head(5)

,updated_at,ad_id,title,type_of_operation,property_type,price,currency,bedrooms_number,full_bathrooms_number,real_state_agency,permalink,thumbnail,address_street,neighborhood,city,state
0,04/22/2021,60924386,"Venta Casa Pedregal, Álvaro Obregón, Ciudad de...",Venta,Casa,32900000,MXN,4,4,maas-inmobiliaria_1527699791800,https://www.inmuebles24.com/propiedades/venta-...,https://img10.naventcdn.com/avisos/18/00/60/92...,Av las fuentes,Jardines del Pedregal,Alvaro Obregón,Distrito Federal
1,04/22/2021,60895016,Magnífica Casa en Una de Las Mejores Colonias ...,Venta,Casa,USD 5500000,USD,3,3,coldwell-banker-invest-asociada-con-id-15246-7_2,https://www.inmuebles24.com/propiedades/magnif...,https://img10.naventcdn.com/avisos/18/00/60/89...,Sierra Gorda,Lomas de Chapultepec,Miguel Hidalgo,Distrito Federal
2,04/22/2021,55875877,Vendo Casa en Lomas de Chapultepec Ciudad de M...,Venta,Casa,61400000,MXN,3,3,trato_directo,https://www.inmuebles24.com/propiedades/vendo-...,https://img10.naventcdn.com/avisos/18/00/55/87...,Lomas de Chapultepec,Lomas de Chapultepec,Miguel Hidalgo,Distrito Federal
3,04/22/2021,59712976,"Residencia en Venta, del Carmen, Coyoacán, Ciu...",Venta,Casa,15180000,MXN,4,4,urbaunidad_1,https://www.inmuebles24.com/propiedades/reside...,https://img10.naventcdn.com/avisos/18/00/59/71...,Hidalgo,Del Carmen,Coyoacán,Distrito Federal
4,04/22/2021,60426612,Casa en Venta en Cuajimalpa,Venta,Casa,7100000,MXN,4,3,placeholder-img.png,https://www.inmuebles24.com/propiedades/casa-e...,https://img10.naventcdn.com/avisos/18/00/60/42...,Cuajimalpa,Cuajimalpa,Cuajimalpa de Morelos,Distrito Federal


### Confirm the integrity of the data and fix errors

In [23]:
count = 0

for x in range(len(propiedades_24_df)):
    research_item = propiedades_24_df['price'][x]
    try:
        int(research_item)
    except:

        print(count, research_item)
    count += 1

1 USD 5500000
130 USD 5499000
131 USD 12000000
137 USD 2700000
182 USD 639000
235 USD 967876
355 USD 3800000
707 USD 1600000
812 USD 830


In [29]:
for x in range(len(propiedades_24_df)):
    price_of_property = propiedades_24_df['price'][x]
    if 'USD' in price_of_property:
        propiedades_24_df['currency'][x] = 'USD'
        proposed_formated_number = price_of_property.replace('USD ', '')
        propiedades_24_df['price'][x] = proposed_formated_number
#         print(f'price_of_property: {propiedades_24_df['price'][x]} - proposed_formated_number: {proposed_formated_number}')

<ipython-input-29-028faaca1472>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  propiedades_24_df['currency'][x] = 'USD'
<ipython-input-29-028faaca1472>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  propiedades_24_df['price'][x] = proposed_formated_number


In [30]:
propiedades_24_df['price'] = pd.to_numeric(propiedades_24_df['price'])

In [31]:
propiedades_24_df['currency'].unique()

array(['MXN', 'USD'], dtype=object)

# ADD UPDATED EXCHANGE RATES - Convert from USD to MXN

In [122]:
# !pip install forex-python

In [32]:
from forex_python.converter import CurrencyRates
c = CurrencyRates()
current_exchange_rates = c.get_rates('USD')   # you can directly call get_rates('USD')

In [33]:
mxn_exchange_rate = current_exchange_rates['MXN']
mxn_exchange_rate

19.9301842935

In [34]:
propiedades_24_df['currency'].value_counts()

MXN    804
USD      9
Name: currency, dtype: int64

In [38]:
len(propiedades_24_df.loc[propiedades_24_df['currency'] == 'USD',:])

0

In [36]:
for x in range(len(propiedades_24_df)):
    if propiedades_24_df['currency'][x] == 'USD':
        propiedades_24_df['price'][x] = float(propiedades_24_df['price'][x]) * mxn_exchange_rate
        propiedades_24_df['currency'][x] = 'MXN'

<ipython-input-36-eec13841f5cb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  propiedades_24_df['price'][x] = float(propiedades_24_df['price'][x]) * mxn_exchange_rate
<ipython-input-36-eec13841f5cb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  propiedades_24_df['currency'][x] = 'MXN'


In [37]:
propiedades_24_df.loc[propiedades_24_df['currency'] == 'USD',:]

,updated_at,ad_id,title,type_of_operation,property_type,price,currency,bedrooms_number,full_bathrooms_number,real_state_agency,permalink,thumbnail,address_street,neighborhood,city,state


## Save the final file with the converted PRICES

In [40]:
# propiedades_24_df.to_csv('csv_files/inmuebles_24_CASAS_CDMX.csv', index = False, encoding = 'utf16')

# -----------------------------------

# Merging the Apartments and Houses dataframes

In [83]:
houses_24_df = pd.read_csv('csv_files/inmuebles_24_CASAS_CDMX.csv', encoding = 'utf16')

In [84]:
apartments_24_df = pd.read_csv('csv_files/inmuebles_24_departamentos_CDMX.csv', encoding = 'utf16')

In [85]:
print(len(houses_24_df))
len(apartments_24_df)

813


1795

In [86]:
full_list_houses_and_apartments = apartments_24_df.append(houses_24_df)
full_list_houses_and_apartments.head()

,updated_at,ad_id,title,type_of_operation,property_type,price,currency,bedrooms_number,full_bathrooms_number,real_state_agency,permalink,thumbnail,address_street,neighborhood,city,state
0,04/22/2021,52913725,Bosque 6060,Venta,Departamento,11312978,MXN,3,not_available,abilia-creando-la-diferencia_1617205333633,https://www.inmuebles24.com/propiedades/bosque...,https://img10.naventcdn.com/listado/RPLISv6.16...,Desierto de los Léones 6060 Ciudad de México,Olivar de los Padres,Alvaro Obregón,Distrito Federal
1,04/22/2021,54456938,Sky Tower Lindavista,Venta,Departamento,3210000,MXN,2,not_available,mp-inmobiliaria_4,https://www.inmuebles24.com/propiedades/sky-to...,https://img10.naventcdn.com/listado/RPLISv6.16...,"Av. Insurgentes Norte. 1346, Lindavista, Ciud...",Lindavista,Gustavo A. Madero,Distrito Federal
2,04/22/2021,55534814,"Último Departamento, Departamento en Venta Tla...",Venta,Departamento,7950000,MXN,3,not_available,emasn-desarrollos_6,https://www.inmuebles24.com/propiedades/ultimo...,https://img10.naventcdn.com/listado/RPLISv6.16...,Club Alemán 37,Arenal Tepepan,Tlalpan,Distrito Federal
3,04/22/2021,60679141,Chicle 160,Venta,Departamento,2052000,MXN,2,not_available,arturo-yanez_1596136396394,https://www.inmuebles24.com/propiedades/chicle...,https://img10.naventcdn.com/listado/RPLISv6.16...,"Chicle 160, Granjas México, Iztacalco, 08400 C...",Granjas México,Iztacalco,Distrito Federal
4,04/22/2021,56515561,Up Santa Fe,Venta,Departamento,3701095,MXN,1,not_available,trato_directo,https://www.inmuebles24.com/propiedades/up-san...,https://img10.naventcdn.com/listado/RPLISv6.16...,"Federal México-Toluca 3009, El Yaqui, 05320 Ci...",Santa Fe,Alvaro Obregón,Distrito Federal


In [87]:
len(full_list_houses_and_apartments)

2608

In [88]:
full_list_houses_and_apartments.property_type.value_counts()

Departamento    1795
Casa             813
Name: property_type, dtype: int64

### Save the final merged file

In [89]:
# full_list_houses_and_apartments.to_csv('csv_files/inmuebles_24_CDMX.csv', index = False, encoding = 'utf16')

## Rename and sort of the columns to match Mercado Libre's dataframe column names

In [90]:
inmuebles_24_full_df = pd.read_csv('csv_files/inmuebles_24_CDMX.csv', encoding = 'utf16')

In [91]:
[x for x in inmuebles_24_full_df.columns]

['updated_at',
 'ad_id',
 'title',
 'type_of_operation',
 'property_type',
 'price',
 'currency',
 'bedrooms_number',
 'full_bathrooms_number',
 'real_state_agency',
 'permalink',
 'thumbnail',
 'address_street',
 'neighborhood',
 'city',
 'state']

In [92]:
inmuebles_24_full_df = inmuebles_24_full_df.rename(columns={'updated_at': 'updated_at',
        'ad_id': 'ID',
        'title': 'Nombre de Publicación',
        'type_of_operation': 'Tipo de Operación',
        'property_type': 'Tipo de propiedad',
        'price': 'Precio',
        'currency': 'Moneda',
        'bedrooms_number': 'Número de recámaras',
        'full_bathrooms_number':'Número de baños completos',
        'real_state_agency': 'Real State Agency?',
        'permalink': 'Link de la publicación',
        'thumbnail':'Foto de publicación',
        'address_street': 'Avenida',
        'neighborhood': 'Colonia',
        'city': 'Ciudad',
        'state': 'Estado'
})

inmuebles_24_full_df.head(5)

,updated_at,ID,Nombre de Publicación,Tipo de Operación,Tipo de propiedad,Precio,Moneda,Número de recámaras,Número de baños completos,Real State Agency?,Link de la publicación,Foto de publicación,Avenida,Colonia,Ciudad,Estado
0,04/22/2021,52913725,Bosque 6060,Venta,Departamento,11312978,MXN,3,not_available,abilia-creando-la-diferencia_1617205333633,https://www.inmuebles24.com/propiedades/bosque...,https://img10.naventcdn.com/listado/RPLISv6.16...,Desierto de los Léones 6060 Ciudad de México,Olivar de los Padres,Alvaro Obregón,Distrito Federal
1,04/22/2021,54456938,Sky Tower Lindavista,Venta,Departamento,3210000,MXN,2,not_available,mp-inmobiliaria_4,https://www.inmuebles24.com/propiedades/sky-to...,https://img10.naventcdn.com/listado/RPLISv6.16...,"Av. Insurgentes Norte. 1346, Lindavista, Ciud...",Lindavista,Gustavo A. Madero,Distrito Federal
2,04/22/2021,55534814,"Último Departamento, Departamento en Venta Tla...",Venta,Departamento,7950000,MXN,3,not_available,emasn-desarrollos_6,https://www.inmuebles24.com/propiedades/ultimo...,https://img10.naventcdn.com/listado/RPLISv6.16...,Club Alemán 37,Arenal Tepepan,Tlalpan,Distrito Federal
3,04/22/2021,60679141,Chicle 160,Venta,Departamento,2052000,MXN,2,not_available,arturo-yanez_1596136396394,https://www.inmuebles24.com/propiedades/chicle...,https://img10.naventcdn.com/listado/RPLISv6.16...,"Chicle 160, Granjas México, Iztacalco, 08400 C...",Granjas México,Iztacalco,Distrito Federal
4,04/22/2021,56515561,Up Santa Fe,Venta,Departamento,3701095,MXN,1,not_available,trato_directo,https://www.inmuebles24.com/propiedades/up-san...,https://img10.naventcdn.com/listado/RPLISv6.16...,"Federal México-Toluca 3009, El Yaqui, 05320 Ci...",Santa Fe,Alvaro Obregón,Distrito Federal


In [93]:
sorted_columns = ['Nombre de Publicación',
 'Estado',
 'Ciudad',
 'Colonia',
 'Avenida',
 'Tipo de propiedad',
 'Tipo de Operación',
 'Precio',
 'Moneda',
 'Número de recámaras',
 'Número de baños completos',
 'Link de la publicación',
 'Foto de publicación',
 'Real State Agency?',
 'ID',
 'updated_at']

In [94]:
inmuebles_24_full_df = inmuebles_24_full_df[sorted_columns]
inmuebles_24_full_df.head()

,Nombre de Publicación,Estado,Ciudad,Colonia,Avenida,Tipo de propiedad,Tipo de Operación,Precio,Moneda,Número de recámaras,Número de baños completos,Link de la publicación,Foto de publicación,Real State Agency?,ID,updated_at
0,Bosque 6060,Distrito Federal,Alvaro Obregón,Olivar de los Padres,Desierto de los Léones 6060 Ciudad de México,Departamento,Venta,11312978,MXN,3,not_available,https://www.inmuebles24.com/propiedades/bosque...,https://img10.naventcdn.com/listado/RPLISv6.16...,abilia-creando-la-diferencia_1617205333633,52913725,04/22/2021
1,Sky Tower Lindavista,Distrito Federal,Gustavo A. Madero,Lindavista,"Av. Insurgentes Norte. 1346, Lindavista, Ciud...",Departamento,Venta,3210000,MXN,2,not_available,https://www.inmuebles24.com/propiedades/sky-to...,https://img10.naventcdn.com/listado/RPLISv6.16...,mp-inmobiliaria_4,54456938,04/22/2021
2,"Último Departamento, Departamento en Venta Tla...",Distrito Federal,Tlalpan,Arenal Tepepan,Club Alemán 37,Departamento,Venta,7950000,MXN,3,not_available,https://www.inmuebles24.com/propiedades/ultimo...,https://img10.naventcdn.com/listado/RPLISv6.16...,emasn-desarrollos_6,55534814,04/22/2021
3,Chicle 160,Distrito Federal,Iztacalco,Granjas México,"Chicle 160, Granjas México, Iztacalco, 08400 C...",Departamento,Venta,2052000,MXN,2,not_available,https://www.inmuebles24.com/propiedades/chicle...,https://img10.naventcdn.com/listado/RPLISv6.16...,arturo-yanez_1596136396394,60679141,04/22/2021
4,Up Santa Fe,Distrito Federal,Alvaro Obregón,Santa Fe,"Federal México-Toluca 3009, El Yaqui, 05320 Ci...",Departamento,Venta,3701095,MXN,1,not_available,https://www.inmuebles24.com/propiedades/up-san...,https://img10.naventcdn.com/listado/RPLISv6.16...,trato_directo,56515561,04/22/2021


In [95]:
inmuebles_24_full_df.shape

(2608, 16)

### Save the final version with the columns renamed and sorted

In [96]:
# inmuebles_24_full_df.to_csv('csv_files/inmuebles_24_CDMX.csv', index = False, encoding = 'utf16')

# -----------------------------------------------------------

# Scrape of each webpage to obtain missing information:
#### 'Condición del Inmueble',
#### 'Número de m2',
#### 'Unidad de medida',
#### 'Latitud',
#### 'Longitud',

In [4]:
inmuebles_24_full_df = pd.read_csv('csv_files/inmuebles_24_CDMX.csv', encoding = 'utf16')

In [5]:
inmuebles_24_full_df.head(2)

,Nombre de Publicación,Estado,Ciudad,Colonia,Avenida,Tipo de propiedad,Tipo de Operación,Precio,Moneda,Número de recámaras,Número de baños completos,Link de la publicación,Foto de publicación,Real State Agency?,ID,updated_at
0,Bosque 6060,Distrito Federal,Alvaro Obregón,Olivar de los Padres,Desierto de los Léones 6060 Ciudad de México,Departamento,Venta,11312978,MXN,3,not_available,https://www.inmuebles24.com/propiedades/bosque...,https://img10.naventcdn.com/listado/RPLISv6.16...,abilia-creando-la-diferencia_1617205333633,52913725,04/22/2021
1,Sky Tower Lindavista,Distrito Federal,Gustavo A. Madero,Lindavista,"Av. Insurgentes Norte. 1346, Lindavista, Ciud...",Departamento,Venta,3210000,MXN,2,not_available,https://www.inmuebles24.com/propiedades/sky-to...,https://img10.naventcdn.com/listado/RPLISv6.16...,mp-inmobiliaria_4,54456938,04/22/2021


In [6]:
inmuebles_24_full_df.shape

(2608, 16)

## THESE ARE SOME TEST FOR SCRAPPING DIFFERENT TYPE OF ADS
### - FOR "DESAROLLO" ADS

In [45]:
url = 'https://www.inmuebles24.com/propiedades/ultimo-departamento-departamento-en-venta-tlalpan-55534814.html'

browser.visit(url)
time.sleep(1.5)
html = browser.html
soup = bs(html, "html.parser")

In [51]:
soup_for_desarrollo = soup.find_all('h3', class_ = 'status-title')[0].get_text()

if soup_for_desarrollo == 'Datos del desarrollo':            

    property_details = soup.find_all('div', class_ = 'row')

    for x in range(len(property_details)):
        category_name = property_details[x].span.get_text(strip = True)
        print(category_name)

        if 'Metros construidos' in category_name:
            category_specs = property_details[x].find_all('span')[1].get_text(strip = True)
            print(category_specs)
            total_area_meters = category_specs.split(" ")[0]
            total_area_meters = int(total_area_meters)
            
            total_area_meters_unit_of_measurement = category_specs.split(" ")[1]
            
            print(total_area_meters_unit_of_measurement)

        if 'Terreno' in category_name:
            category_specs = property_details[x].find_all('span')[1].get_text(strip = True)
            print(category_specs)
            land_size = category_specs.split(" ")[0]
            land_size = int(land_size)
            
            land_size_unit_of_measurement = category_specs.split(" ")[1]
            
            print(land_size_unit_of_measurement)

Entrega
Metros construidos
185 m²
m²
Unidades
Recámaras
Terreno
335 m²
m²
Estacionamientos
Precio desde


In [80]:
#Normal Ad

soup.find_all('li', class_ = 'icon-feature')

[]

### - FOR "FINALIZADA" ADS

In [81]:
url = 'https://www.inmuebles24.com/propiedades/departamentos-en-venta-en-calle-amores-ciudad-de-60806063.html'

browser.visit(url)
time.sleep(1.5)
html = browser.html
soup = bs(html, "html.parser")

In [82]:
# Finished Ad

soup.find_all('div', class_ = 'offline-content')[0].p.get_text(strip = True)

'Aviso finalizado'

In [83]:
#Normal Ad

soup.find_all('li', class_ = 'icon-feature')

[<li class="icon-feature">
 <i class="icon-stotal"></i>
 				
 					87
 					
 						m² Terreno
 					
 				
 			</li>,
 <li class="icon-feature">
 <i class="icon-bano"></i>
 				
 					2
 					
 						
 							Baños
 						
 					
 				
 			</li>,
 <li class="icon-feature">
 <i class="icon-cochera"></i>
 				
 					2
 					
 						
 							Estacionamientos
 						
 					
 				
 			</li>,
 <li class="icon-feature">
 <i class="icon-dormitorio"></i>
 				
 					2
 					
 						
 							Recámaras
 						
 					
 				
 			</li>,
 <li class="icon-feature">
 <i class="icon-antiguedad"></i>
 						
 							A estrenar
 						
 					</li>]

In [84]:
#Desarrollo Ad

soup.find_all('h3', class_ = 'status-title')[0].get_text()

IndexError: list index out of range

## SCRAPE FOR NORMAL ANNOUNCEMENT

In [10]:
url = 'https://www.inmuebles24.com/propiedades/departamento-en-venta-en-la-colonia-del-valle-ciudad-60006486.html'

browser.visit(url)
time.sleep(1.5)
html = browser.html
soup = bs(html, "html.parser")

In [38]:
ad_icon_features = soup.find_all('li', class_ = 'icon-feature')

for x in range(len(ad_icon_features)):
    feature = ad_icon_features[x].get_text(strip = True).replace('\n' , ' ').replace('\t' , '')
    
#     print(feature)
    
    if 'Terreno' in feature:
        land_size = feature.split(" ")[0]
        land_size = int(land_size)
        
        land_size_unit_of_measurement = feature.split(" ")[2]
        
    if 'Construido' in feature:
        total_area_meters = feature.split(" ")[0]
        total_area_meters = int(total_area_meters)  
        
        total_area_meters_unit_of_measurement = feature.split(" ")[2]        
    
    if 'Antigüedad' in feature:
        item_condition = feature.split(" ")[0]
        
    
if total_area_meters >= land_size:
    property_meters_size = total_area_meters
    property_measure_unit = total_area_meters_unit_of_measurement
    
elif land_size > total_area_meters:
    property_meters_size = land_size
    property_measure_unit = land_size_unit_of_measurement

print(f'Tamaño de la propiedad: {property_meters_size} m², Measure_unit: {property_measure_unit}, item_condition: {item_condition} years')
    



Tamaño de la propiedad: 103 m², Measure_unit: m², item_condition: 30 years


In [23]:
inmuebles_24_full_df.loc[inmuebles_24_full_df['ID'] == 60006486]

,Nombre de Publicación,Estado,Ciudad,Colonia,Avenida,Tipo de propiedad,Tipo de Operación,Precio,Moneda,Número de recámaras,Número de baños completos,Link de la publicación,Foto de publicación,Real State Agency?,ID,updated_at
9,"Departamento en Venta en La Colonia del Valle,...",Distrito Federal,Benito Juárez,Del Valle,Luz Savignon,Departamento,Venta,4300000,MXN,3,2,https://www.inmuebles24.com/propiedades/depart...,https://img10.naventcdn.com/avisos/18/00/60/00...,trato_directo,60006486,04/22/2021


In [90]:
browser.quit()

# After doing the necessary tests, we set up the code that will run on a for loop to extract the info on several pages

In [2]:
executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)

[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/89.0.4389.23/chromedriver_win32.zip


[WDM] - Driver has been saved in cache [C:\Users\Intel NUC\.wdm\drivers\chromedriver\win32\89.0.4389.23]


In [ ]:
# for x in range(0, 500): 24-Abr-2021 1:00 PM
# for x in range(500, 1000): 24-Abr-2021 2:20 PM
# for x in range(1000, 1500): 24-Abr-2021 3:45 PM
# for x in range(1500, 1861): 24-Abr-2021 4:19 PM
# for x in range(1861, 2600): 24-Abr-2021 5:28 PM

In [59]:
import time

latitude_list = []
longitude_list = []
property_id_list = []
item_condition_list = []

land_size_list = []
land_size_unit_of_measurement_list = []
total_area_meters_list = []
total_area_meters_unit_of_measurement_list = []

count_number = 0
image_details_coordinates_list = []
url_list = []


for x in range(2600, len(inmuebles_24_full_df)): 

    latitude = ''
    longitude = ''
    item_condition = ''
    land_size = 0
    land_size_unit_of_measurement = ''
    total_area_meters = 0
    total_area_meters_unit_of_measurement = ''
    
    count_number += 1

    url = inmuebles_24_full_df['Link de la publicación'][x]

    browser.visit(url)
    time.sleep(2.5)
    html = browser.html
    soup = bs(html, "html.parser")
    
#    URL list

    url_list.append(url)    
    
#     PROPERTY ID

    property_id = inmuebles_24_full_df['ID'][x]    
    property_id_list.append(property_id)
    
#   PROPERTY COORDINATES:

    try:
        map_image = soup.find('div', class_ = 'article-map')
        
        image_details = map_image.img['src'].split("&")


        for j in range(len(image_details)):
            if 'markers' in image_details[j]:
                image_details_coordinates = image_details[j].replace('markers=', '').split(",")            
                latitude = image_details_coordinates[0]
                longitude = image_details_coordinates[1]            

        if len(image_details_coordinates_list) != count_number:
            for d in range(len(image_details)):
                if 'center' in image_details[d]:
                    image_details_coordinates = image_details[d].replace('center=', '').split(",")
                    latitude = image_details_coordinates[0].split("?")[1]
                    longitude = image_details_coordinates[1]


    
    except:        
        image_details_coordinates = 'hidden_address'
        latitude = 'hidden_address'
        longitude = 'hidden_address'
        
    
    image_details_coordinates_list.append(image_details_coordinates)
    latitude_list.append(latitude)
    longitude_list.append(longitude)
                
#     PROPERTY SIZE:

    #Desarrollo Ad

    try:
        soup_for_desarrollo = soup.find_all('h3', class_ = 'status-title')[0].get_text()
        
        if soup_for_desarrollo == 'Datos del desarrollo':
            
            item_condition = 'new'

            property_details = soup.find_all('div', class_ = 'row')

            for i in range(len(property_details)):                
                category_name = property_details[i].span.get_text(strip = True)

                if 'Metros construidos' in category_name:
                    category_specs = property_details[i].find_all('span')[1].get_text(strip = True)
                    total_area_meters = category_specs.split(" ")[0]
                    total_area_meters = int(total_area_meters)

                    total_area_meters_unit_of_measurement = category_specs.split(" ")[-1]


                if 'Terreno' in category_name:
                    category_specs = property_details[i].find_all('span')[1].get_text(strip = True)
                    land_size = category_specs.split(" ")[0]
                    land_size = int(land_size)

                    land_size_unit_of_measurement = category_specs.split(" ")[-1]
                
    except:
        
    #For any other kind of ad:
    
        ad_icon_features = soup.find_all('li', class_ = 'icon-feature')

        for d in range(len(ad_icon_features)):
            
            feature = ad_icon_features[d].get_text(strip = True).replace('\n' , ' ').replace('\t' , '')

            if 'Terreno' in feature:
                land_size = feature.split(" ")[0]
                land_size = int(land_size)

                land_size_unit_of_measurement = feature.split(" ")[2]

            if 'Construido' in feature:
                total_area_meters = feature.split(" ")[0]
                total_area_meters = int(total_area_meters)  

                total_area_meters_unit_of_measurement = feature.split(" ")[2]        

            if 'Antigüedad' or 'estrenar' in feature:
                item_condition = feature
                
  
    land_size_list.append(land_size)
    land_size_unit_of_measurement_list.append(land_size_unit_of_measurement)
    total_area_meters_list.append(total_area_meters)
    total_area_meters_unit_of_measurement_list.append(total_area_meters_unit_of_measurement)
    
    item_condition_list.append(item_condition)
    
    print(f'SUCCESFUL - Index_number: {x} - {url}')
    print(f'land_size: {land_size} m², total_area_meters: {total_area_meters} m², Measure_unit: {land_size_unit_of_measurement, total_area_meters_unit_of_measurement}, Antigüedad: {item_condition}')
    print(latitude, longitude)
    print("")    


SUCCESFUL - Index_number: 2600 - https://www.inmuebles24.com/propiedades/casa-en-condominio-horizontal-tlalpan-60316845.html
land_size: 63 m², total_area_meters: 164 m², Measure_unit: ('m²', 'm²'), Antigüedad: 20  Antigüedad
19.277636000000000 -99.205033399999990

SUCCESFUL - Index_number: 2601 - https://www.inmuebles24.com/propiedades/se-vende-hermosa-casa-en-calle-cerrada-remodelada-y-60511213.html
land_size: 189 m², total_area_meters: 170 m², Measure_unit: ('m²', 'm²'), Antigüedad: 30  Antigüedad
19.334404400000000 -99.123188300000000

SUCCESFUL - Index_number: 2602 - https://www.inmuebles24.com/propiedades/bonita-casa-en-romero-rubio-59547356.html
land_size: 179 m², total_area_meters: 179 m², Measure_unit: ('m²', 'm²'), Antigüedad: 2   Recámaras
19.442391300000000 -99.096379799999994

SUCCESFUL - Index_number: 2603 - https://www.inmuebles24.com/propiedades/propiedad-de-prueba-59213617.html
land_size: 3 m², total_area_meters: 3 m², Measure_unit: ('m²', 'm²'), Antigüedad: 3   Medios 

### Debugger code

In [60]:
fields = [latitude_list,
    longitude_list,
    property_id_list,          
    land_size_list,
    land_size_unit_of_measurement_list,
    total_area_meters_list,
    total_area_meters_unit_of_measurement_list,          
    item_condition_list,          
    url_list]

for x in range(len(fields)):
    print(f'field {x+1}, length: {len(fields[x])}')

field 1, length: 8
field 2, length: 8
field 3, length: 8
field 4, length: 8
field 5, length: 8
field 6, length: 8
field 7, length: 8
field 8, length: 8
field 9, length: 8


## Creation of a dataframe using the information scrapped

In [61]:
details_from_properties_24_df = pd.DataFrame({
    "ID": property_id_list,
    "Condición del Inmueble": item_condition_list,   
    'land_size': land_size_list,
    'land_size_unit': land_size_unit_of_measurement_list,
    'total_area': total_area_meters_list,
    'total_area_unit': total_area_meters_unit_of_measurement_list,
    "Latitud": latitude_list,
    "Longitud": longitude_list,
    "URL_FOR_REF": url_list
})

details_from_properties_24_df.head()

,ID,Condición del Inmueble,land_size,land_size_unit,total_area,total_area_unit,Latitud,Longitud,URL_FOR_REF
0,60316845,20 Antigüedad,63,m²,164,m²,19.277636000000000,-99.205033399999990,https://www.inmuebles24.com/propiedades/casa-e...
1,60511213,30 Antigüedad,189,m²,170,m²,19.334404400000000,-99.123188300000000,https://www.inmuebles24.com/propiedades/se-ven...
2,59547356,2 Recámaras,179,m²,179,m²,19.442391300000000,-99.096379799999994,https://www.inmuebles24.com/propiedades/bonita...
3,59213617,3 Medios Baños,3,m²,3,m²,19.418310800000000,-99.164471800000001,https://www.inmuebles24.com/propiedades/propie...
4,60025089,1 Medio baño,180,m²,120,m²,19.435716688379503,-99.131784439086914,https://www.inmuebles24.com/propiedades/casa-l...


In [62]:
details_from_properties_24_df.tail()

,ID,Condición del Inmueble,land_size,land_size_unit,total_area,total_area_unit,Latitud,Longitud,URL_FOR_REF
3,59213617,3 Medios Baños,3,m²,3,m²,19.418310800000000,-99.164471800000001,https://www.inmuebles24.com/propiedades/propie...
4,60025089,1 Medio baño,180,m²,120,m²,19.435716688379503,-99.131784439086914,https://www.inmuebles24.com/propiedades/casa-l...
5,59138274,8 Antigüedad,30000,m²,300000,m²,19.407202000000001,-99.167871099999999,https://www.inmuebles24.com/propiedades/casa-d...
6,60387713,10 Antigüedad,500,m²,250,m²,19.361845599999998,-99.186225800000002,https://www.inmuebles24.com/propiedades/casa-e...
7,59145026,3 Recámaras,0,,0,,19.411725362792935,-99.109308201280370,https://www.inmuebles24.com/propiedades/gran-o...


## Save the scrapped file

In [67]:
# details_from_properties_24_df.to_csv('csv_files/DETAILS_inmuebles_24_CASAS_CDMX.csv', index = False, encoding = 'latin1')

In [68]:
browser.quit()